# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
前章で作成したマルコフモデルの様な「ある単語から次の単語を予測するモデル」をニューラルネットワークを用いて作成する。

本章では、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import random
from typing import List

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
textfile = 'data/jawiki.txt'
with open(textfile) as f:
    data = f.readlines()

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。\n',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。\n',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。\n',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜

多すぎるので減らす

In [4]:
data = data[:1000]

textfile = 'data/jawiki_1000.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence)


---

## サブワード分割

トークン化の手法。

NNでも、マルコフモデル同様、文章をトークン化して学習を行う。  
NNでは文字列をそのまま扱えないので、各トークンにID=クラスラベルを割り当てて、そのシーケンスとして文章を扱う。

前章とは異なり、サブワード分割という新たな手法を使う。ちゃんと理由もある。

1トークンあたりの文字数が多いことは、いくつかのデメリットを生む。  
例えば、トークンの種類が多くなること。深層学習を用いた言語モデルはトークンの種類に比例してパラメータの数が増え、学習が困難になる。  
それから、未知語が増えること。長いトークンは、それだけ多くの情報を持った限定的な言葉ということとなり、表現力が落ちる。またデータセットに存在しない言葉を扱うことが困難になる。

サブワード分割は、これらの問題を解決する。  
サブワードと呼ばれる、単語よりも小さな単位に分割することで、1トークンあたりの文字数を減らす。

この手法では、データセットから頻出する文字の並びを学習し、その並びをトークンとして分割する。学習された、トークン化を行うモデルをトークナイザと呼ぶ。  
データセットに合ったトークン化が可能。また語彙数を指定することも可能。指定した語彙数に収まるまで細かく分割してくれる。

ちなみに、ChatGPTが使っているトークナイザはこれ: [OpenAI Platform](https://platform.openai.com/tokenizer)

少し余談。

前章でトークン化について以下のように説明した。

> トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。

ここで、そもそも言語モデルは単語の確率を扱うものなので、絶対にトークン=単語にならないとおかしいとも考えられる。  
ただ、実は文章生成においては、単語を最小単位にしなければならない理由はない。極端な話、文字を最小単位として文章を文字の並びと見てもいい訳だ。なんらかのシーケンスとできればそれで充分なのだ。実際、現在有名な言語モデルの多くは単語を最小単位としていない。

言語モデルは単語の並びに確率を割り当てるモデルだ。単語を最小単位とするモデルだ。  
もしこの定義に厳格になるのであれば、現在有名な多くの言語モデルは言語モデルと呼べないのかもしれんね。しらんけど。

閑話休題。実際にやってみよう。  
sentencepieceというライブラリを用いる。

In [5]:
import sentencepiece as spm

### 学習

語彙数とデータセット（テキストファイル）を指定して、学習させる。

In [ ]:
tokenizer_prefix = 'models/tokenizer_jawiki_1000' # トークナイザのモデル名
vocab_size = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=textfile, # データセット
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size
)

### トークン化

学習したモデルにテキストを突っ込むとトークン化してくれる。

In [7]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model') # モデル読み込み
text = '今日はいい天気だ'
ids = sp.encode(text)
ids

[11, 1993, 6, 2317, 681, 343, 263]

テキストがトークン化され、ID列として取得できる。  
`out_type`を指定すると文字列のリストが取得できる。

In [8]:
sp.encode(text, out_type=str)

['▁', '今日', 'は', 'いい', '天', '気', 'だ']

アンダーバーは空白を意味する。初めにアンダーバーが付くのは仕様。

ID列を文字列に戻すこともできる。

In [9]:
sp.decode(ids)

'今日はいい天気だ'

全てのデータをトークン化（ID化）しよう。  
文章のリスト: `List[str]`を`sp.encode()`に与えるとID列のリスト: `List[List[int]]`が返ってくる。

In [11]:
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 18, 6254, 54, 1057, 58, 1685, 79, 122, 17]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [12]:
Ids = List[int]
class TextDataset(Dataset):
    def __init__(self, data_ids: List[Ids]):
        self.x, self.y = [], []
        for ids in data_ids:
            self.x += ids[:-1]
            self.y += ids[1:]
        self.n_data = len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.n_data

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 243959
tensor([5462, 5535,  342,   16, 3452,   15,  269,  862,   64,  281, 1107,  821,
        2144, 1806, 7991,   43,    7,  137,  672,   30, 3882, 4052,   34,    6,
          62,  277,   12,  117,  816, 2013,   40, 2910,  666,   21,    5,   52,
           9, 1902,    6,  270,  781,  215,   34,  320,    3, 3105,  504, 1471,
         146,    4, 2992,  167, 4274, 3264,   24,    5, 5147,  306, 1288, 2256,
         439,    8,  128,  583])
tensor([ 131, 4199,    4,   83,   53, 1233,    6,  574, 5746, 2237,    3, 5365,
           9,  441,    9,  454,   20, 7988,  103, 1117, 4483,    8, 1285,   20,
         603,  792,    3,   82, 6449,  282,  559,   14,    4,  164, 1263,  686,
        3883,  220,   52,    7, 1115,  222,   57, 2057,  751, 2403,    6, 2840,
        6195, 1741,    4, 7363,    5,   88,    4,  243,   16,  777,   10,  128,
          22, 1708,  537, 4308])



---

## モデル構築

ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDは入力時にone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルとする。  
つまりこのタスクは単語の分類問題とも言える。

In [13]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()
        self.n_vocab = n_vocab
        self.net = nn.Sequential(
            nn.Linear(n_vocab, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x): # x: (batch_size, n_vocab)
        x = F.one_hot(x, self.n_vocab).to(torch.float32) # one-hotベクトルに
        y = self.net(x)
        return y # (batch_size, n_vocab)


---

## 学習

損失関数に交差エントロピーを設定し、通常の分類モデルと同じように学習する。

In [14]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(1, n_epochs + 1):
        for x, t in dataloader:
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [15]:
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [16]:
train(model, optimizer, n_epochs=20, prog_unit=2)

  1-2/20: ######################################## 100% [00:00:29.23] loss: 6.34186 
  3-4/20: ######################################## 100% [00:00:25.83] loss: 5.01536 
  5-6/20: ######################################## 100% [00:00:25.52] loss: 4.45553 
  7-8/20: ######################################## 100% [00:00:25.62] loss: 4.27914 
 9-10/20: ######################################## 100% [00:00:25.52] loss: 4.21797 
11-12/20: ######################################## 100% [00:00:25.72] loss: 4.18796 
13-14/20: ######################################## 100% [00:00:26.15] loss: 4.16931 
15-16/20: ######################################## 100% [00:00:25.45] loss: 4.15577 
17-18/20: ######################################## 100% [00:00:25.86] loss: 4.14509 
19-20/20: ######################################## 100% [00:00:25.71] loss: 4.13704 



---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。ある単語からの次の単語の予測を繰り返すことで文章を生成する。初めの単語だけはこちらで指定する。  
以下の条件を満たすまで単語の生成を続ける。
- 単語数が指定した限界に達する
- 句点が出力される

In [17]:
end_id = sp.piece_to_id('。') # 句点のID

def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    probs = F.softmax(y, dim=-1) # 確率分布に変換
    token, = random.choices(range(n_vocab), weights=probs) # サンプリング
    return token

def generate_sentence(
    model: nn.Module,
    start_word: str,
    max_len: int = 30
) -> str:
    model.eval()
    token_id = sp.piece_to_id(start_word)
    token_ids = [token_id]

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != end_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        token_id = token_sampling(y) # サンプリング
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [18]:
start_words = ['今日', '地球', '科学', '人']
for start_word in start_words:
    print(generate_sentence(model, start_word))

今日の練習にはジョン手が、桑田靖子)、強制改革はリンジリアの手法を引用の歴史記述を参照)正五位下に地
地球に招き、上に書かれていたが、であるローマ帝国の移動の意味の金水にアルバートが争が行われた。
科学における建築的であるも広東省分科連射程、ピアノと照準惑星で吉はマビヨンが寝め、簡による捜索硬化症
人は、コンスタンティヌス140億ドルのうち、ラント海 ⁇ 山県知する。


マルコフモデル同様、直前の単語のみを予測に用いているため、不自然な文章が多く生成される。


---

## 埋め込み層

*Embedding Layer*

NN内部における、単語IDに対する「one-hotベクトル化→線形変換」を行う部分をまとめて**埋め込み層**と表す。  
そもそも、これらの処理は単語IDを受け取って対応するベクトルを出力すること=**単語のベクトル化**と同義である。自然言語処理の世界では単語のベクトル化を**単語の埋め込み**と表現する。

線形変換を行う全結合層は各単語の埋め込み表現（単語ベクトル）を保有していることになる。それらは重みから確認できる。

In [19]:
class Embedding(nn.Module):
    def __init__(self, n_vocab: int, embed_dim: int):
        super().__init__()
        self.n_vocab = n_vocab
        self.fc = nn.Linear(n_vocab, embed_dim, bias=False)

    def forward(self, x):
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
        h = self.fc(x)
        return h

embedding = Embedding(n_vocab=3, embed_dim=5)

x = torch.tensor([0, 1, 2])
h = embedding(x)

print(h, '\n') # 埋め込み層からの出力
print(embedding.fc.weight.T) # 全結合層の重み

tensor([[ 0.5257,  0.2516, -0.4690, -0.1184, -0.3921],
        [-0.2539,  0.4104,  0.3077,  0.5058,  0.5663],
        [-0.5706, -0.1307,  0.4886, -0.4432,  0.0395]], grad_fn=<MmBackward0>) 

tensor([[ 0.5257,  0.2516, -0.4690, -0.1184, -0.3921],
        [-0.2539,  0.4104,  0.3077,  0.5058,  0.5663],
        [-0.5706, -0.1307,  0.4886, -0.4432,  0.0395]],
       grad_fn=<PermuteBackward0>)


`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
[Embedding — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [20]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, hidden_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Embedding(n_vocab, hidden_size), # Embedding層
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x):
        y = self.net(x)
        return y

実際に、言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させることもある。word2vecなどがこれに該当し、これについてはおまけの章で取り上げる。


---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### *Unknown*

未知語を意味するトークン。  
推論時に学習データに含まれなかった単語に出会ったときに対応できるようになる。学習データ内での出現回数が少ない単語も未知語として扱うことがある。

### *Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### *End of Sentence*

文章の終わりを意味するトークン。  
先程の例では句点が出た場合に生成を止めたが、本来句点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが直接触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けたりすることが多い。

sentencepieceでは上記3つのトークンがID0, 1, 2にデフォルトで設定されている。

In [21]:
n_vocab = len(sp)
print('ID0:', sp.id_to_piece(0)) # Unknown
print('ID1:', sp.id_to_piece(1)) # Begin of Sentence
print('ID2:', sp.id_to_piece(2)) # End of Sentence

ID0: <unk>
ID1: <s>
ID2: </s>


では、トークン列にBOSとEOSを追加して学習させてみよう。

In [22]:
bos_id = sp.bos_id() # BOSのID
eos_id = sp.eos_id() # EOSのID
for ids in data_ids:
    ids.insert(0, bos_id) # 先頭にBOSを追加
    ids.append(eos_id) # 末尾にEOSを追加

In [23]:
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = LanguageModel(n_vocab, 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [24]:
train(model, optimizer, n_epochs=10)

 1/10:                                            0% [00:00:00.13] loss: 8.09257 

 1/10: ######################################## 100% [00:00:12.63] loss: 7.01418 
 2/10: ######################################## 100% [00:00:12.56] loss: 5.28972 
 3/10: ######################################## 100% [00:00:12.57] loss: 4.93946 
 4/10: ######################################## 100% [00:00:12.52] loss: 4.79246 
 5/10: ######################################## 100% [00:00:12.62] loss: 4.70954 
 6/10: ######################################## 100% [00:00:12.67] loss: 4.65897 
 7/10: ######################################## 100% [00:00:12.48] loss: 4.62054 
 8/10: ######################################## 100% [00:00:12.73] loss: 4.59307 
 9/10: ######################################## 100% [00:00:12.48] loss: 4.57220 
10/10: ######################################## 100% [00:00:12.69] loss: 4.55528 


では文章を生成する。初めのトークンはBOSとし、EOSが出力されるまで生成を続ける。

In [25]:
unk_id = sp.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    y[unk_id] = -torch.inf # UNKがサンプリングされる確率を0にする
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

def generate_sentence(model, max_len=50):
    model.eval()
    token_id = bos_id
    token_ids = []

    for _ in range(max_len):
        x = torch.tensor(token_id).unsqueeze(0).to(device)
        y = model(x)[0]
        token_id = token_sampling(y)
        if token_id == eos_id: # EOSが生成されたら終了
            break
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [26]:
for _ in range(5):
    print(generate_sentence(model))

μ Worletery) た「実施した。父が戦死したため、下」と見た。出来るがサンバ、さらに9年前後方向にかけ離れた。その場でまたは「眼で
NのEenkyfrichast 18日キープニングに新線mで死去したと直接目標値を支配に儒教えんの腕の代行された。スウィッシング・ブレンターノを中心として成立した
fe(カ県」(田所有んだことの封じゃ勝ってもいる。2008年に実対岸を見事になる。ガ・モール学派の経済的リスボイドもほとんど歩による演奏会報』を参考にして初優勝
アドシングルでは佐伯(11シーズンは最後の方向がCBAも編集作業中に収される。スサービスUMP1.1日に、佐した時に使用している。1948月168年連れて計画HPエルよりや
映画・実はフリースウィの老のデビューイベントとして天皇の線路を、川に沿って任音高角絞られてきた。
